In [2]:
import csv
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib import dates
import datetime
from datetime import timedelta

In [3]:
def import_files(file_path, data_type, airport, date):

    filename = '{0}/{1}/{2}/{1}_{3}.txt'.format(file_path, data_type, airport, date) 

    if not os.path.isfile(filename):
        return -1

    data = pd.read_csv(filename)

    data['TIME'] = data['TIME'].astype(str) # convert time to string so '--' can be replaced

    data['TIME'] = data['TIME'].str.replace('--', '00') # replace null seconds with 00   

    data['TIME'] = pd.to_datetime(data['TIME'], format='%Y%m%d%H%M%S')

    data.set_index('TIME', inplace=True)

    return(data)

In [31]:
def main():

    #---------------------------------------------------------------------
    # 01. Settings
    #---------------------------------------------------------------------

    file_path = 'D:/MetDB/'
    out_path = 'D:/Mode-S_altitude/Mode-S_day_min'

    period = 'winter' # summer or winter

    if period == 'summer':
        start_date = datetime.date(2021,7,10)
        #end_date = datetime.date(2021,7,15)
        end_date = datetime.date(2021,8,10)

    if period == 'winter':
        start_date = datetime.date(2022,1,1)
        end_date = datetime.date(2022,1,31)

    date_list = [start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)]
    date_list = [date_obj.strftime('%Y%m%d') for date_obj in date_list]

    #airport_name_list = ['Bristol']
    
    airport_name_list = ['Heathrow', \
                     'Gatwick', \
                     'Manchester', \
                     'Stansted', \
                     'Edinburgh', \
                     'Birmingham', \
                     'Bristol', \
                     'Glasgow', \
                     'Aberdeen', \
                     'EastMidlands', \
                     'LondonCity', \
                     'BelfastInt', \
                     'Newcastle', \
                     'LeedsBradford', \
                     'Liverpool',\
                     'Cardiff']
    
    #---------------------------------------------------------------------    
    # 02. Loop through airports/dates and plot altitude
    #--------------------------------------------------------------------- 
    ptGeoms = []
    for airport in airport_name_list:

        concat_amdar = pd.DataFrame()
        concat_modes = pd.DataFrame()
        concat_modes_day_min = pd.DataFrame()

        for date in date_list:

            print('Loading data for: {0} {1}'.format(airport, date))

            data_amdar = import_files(file_path, 'AMDARS', airport, date)
            data_modes = import_files(file_path, 'MODE-S', airport, date)
            
            #Find row containing daily min
            data_modes_day_min = data_modes[data_modes.GNSS_ALTD==data_modes.GNSS_ALTD.min()]

            concat_amdar = pd.concat([concat_amdar, data_amdar])
            concat_modes = pd.concat([concat_modes, data_modes])
            
            concat_modes_day_min = pd.concat([concat_modes_day_min, data_modes_day_min])
 
        # Create dataframe of daily minimum Mode-S altitudes 
        concat_modes_day_min.reset_index(inplace=True)         
        concat_modes_day_min['TIME'] = concat_modes_day_min['TIME'].dt.strftime('%Y%m%d')    
        concat_modes_day_min = concat_modes_day_min.drop_duplicates(['TIME'], keep='first')
        #concat_modes_day_min.to_csv(os.path.join(out_path, "{0}_{1}_LL.csv".format(airport, period)))
        
        '''
        # Convert day min lat/lon to British Grid - SLOW WAY TO DO IT
        gt = 'OSGB_1936_To_WGS_1984_7'  
        inputSRS = arcpy.SpatialReference(4326) # GCS WGS84    
        outputSRS = arcpy.SpatialReference(7405) # British National Grid 
        #ptGeoms = []
        
        for idx, row in concat_modes_day_min.iterrows():
            pt = arcpy.Point() # create point object 
            pt.Y = row['LAT']
            pt.X = row['LON']
            print("Input XY: {0} {1}".format(pt.X, pt.Y))
            ptGeoms.append(arcpy.PointGeometry(pt,inputSRS ))
            arcpy.CopyFeatures_management(ptGeoms, 'base')
            arcpy.Project_management('base',gt, outputSRS)
            
        #with arcpy.da.SearchCursor("WGS_1984_To_OSGB_1936",['SHAPE@X','SHAPE@Y']) as cursor:
            #for row in cursor:
                #print(row[0])
                #print(row[1])
        '''   
        # Convert day min lat/lon to British Grid 
        gt = 'OSGB_1936_To_WGS_1984_7'  
        inputSRS = arcpy.SpatialReference(4326) # GCS WGS84    
        outputSRS = arcpy.SpatialReference(7405) # British National Grid       
        
        for idx, row in concat_modes_day_min.iterrows():
            pt = arcpy.Point() # create point object 
            pt.Y = row['LAT']
            pt.X = row['LON']
            #print("Input XY: {0} {1}".format(pt.X, pt.Y))
            ptgeo = arcpy.PointGeometry(pt, inputSRS)
            ptgeo1 = ptgeo.projectAs(outputSRS, gt)
            pt1 = ptgeo1.lastPoint
            #print("Output XY: {0} {1}".format(pt1.X, pt1.Y))
            new_lon = round(pt1.X,2)
            new_lat = round(pt1.Y,2)
            concat_modes_day_min.loc[idx,'LAT'] = new_lat
            concat_modes_day_min.loc[idx,'LON'] = new_lon
        
        print(concat_modes_day_min)
        concat_modes_day_min.to_csv(os.path.join(out_path, "{0}_{1}_day_min_BG.csv".format(airport, period)),index=False)
       

In [32]:
main()

Loading data for: Heathrow 20220101
Loading data for: Heathrow 20220102
Loading data for: Heathrow 20220103
Loading data for: Heathrow 20220104
Loading data for: Heathrow 20220105
Loading data for: Heathrow 20220106
Loading data for: Heathrow 20220107
Loading data for: Heathrow 20220108
Loading data for: Heathrow 20220109
Loading data for: Heathrow 20220110
Loading data for: Heathrow 20220111
Loading data for: Heathrow 20220112
Loading data for: Heathrow 20220113
Loading data for: Heathrow 20220114
Loading data for: Heathrow 20220115
Loading data for: Heathrow 20220116
Loading data for: Heathrow 20220117
Loading data for: Heathrow 20220118
Loading data for: Heathrow 20220119
Loading data for: Heathrow 20220120
Loading data for: Heathrow 20220121
Loading data for: Heathrow 20220122
Loading data for: Heathrow 20220123
Loading data for: Heathrow 20220124
Loading data for: Heathrow 20220125
Loading data for: Heathrow 20220126
Loading data for: Heathrow 20220127
Loading data for: Heathrow 2

36  20220131  b'407512  '  385126.32  383036.94       46.0  160.01999
Loading data for: Stansted 20220101
Loading data for: Stansted 20220102
Loading data for: Stansted 20220103
Loading data for: Stansted 20220104
Loading data for: Stansted 20220105
Loading data for: Stansted 20220106
Loading data for: Stansted 20220107
Loading data for: Stansted 20220108
Loading data for: Stansted 20220109
Loading data for: Stansted 20220110
Loading data for: Stansted 20220111
Loading data for: Stansted 20220112
Loading data for: Stansted 20220113
Loading data for: Stansted 20220114
Loading data for: Stansted 20220115
Loading data for: Stansted 20220116
Loading data for: Stansted 20220117
Loading data for: Stansted 20220118
Loading data for: Stansted 20220119
Loading data for: Stansted 20220120
Loading data for: Stansted 20220121
Loading data for: Stansted 20220122
Loading data for: Stansted 20220123
Loading data for: Stansted 20220124
Loading data for: Stansted 20220125
Loading data for: Stansted 202

46  20220131  b'407A05  '  279957.58  408681.02     1661.0  1752.59998
Loading data for: Bristol 20220101
Loading data for: Bristol 20220102
Loading data for: Bristol 20220103
Loading data for: Bristol 20220104
Loading data for: Bristol 20220105
Loading data for: Bristol 20220106
Loading data for: Bristol 20220107
Loading data for: Bristol 20220108
Loading data for: Bristol 20220109
Loading data for: Bristol 20220110
Loading data for: Bristol 20220111
Loading data for: Bristol 20220112
Loading data for: Bristol 20220113
Loading data for: Bristol 20220114
Loading data for: Bristol 20220115
Loading data for: Bristol 20220116
Loading data for: Bristol 20220117
Loading data for: Bristol 20220118
Loading data for: Bristol 20220119
Loading data for: Bristol 20220120
Loading data for: Bristol 20220121
Loading data for: Bristol 20220122
Loading data for: Bristol 20220123
Loading data for: Bristol 20220124
Loading data for: Bristol 20220125
Loading data for: Bristol 20220126
Loading data for: B

53  20220131  b'484966  '  811310.97  388576.62       38.0   91.43999
Loading data for: EastMidlands 20220101
Loading data for: EastMidlands 20220102
Loading data for: EastMidlands 20220103
Loading data for: EastMidlands 20220104
Loading data for: EastMidlands 20220105
Loading data for: EastMidlands 20220106
Loading data for: EastMidlands 20220107
Loading data for: EastMidlands 20220108
Loading data for: EastMidlands 20220109
Loading data for: EastMidlands 20220110
Loading data for: EastMidlands 20220111
Loading data for: EastMidlands 20220112
Loading data for: EastMidlands 20220113
Loading data for: EastMidlands 20220114
Loading data for: EastMidlands 20220115
Loading data for: EastMidlands 20220116
Loading data for: EastMidlands 20220117
Loading data for: EastMidlands 20220118
Loading data for: EastMidlands 20220119
Loading data for: EastMidlands 20220120
Loading data for: EastMidlands 20220121
Loading data for: EastMidlands 20220122
Loading data for: EastMidlands 20220123
Loading da

37  20220131  b'48C238  '  437161.19  426073.54      373.0  502.91998
Loading data for: Liverpool 20220101
Loading data for: Liverpool 20220102
Loading data for: Liverpool 20220103
Loading data for: Liverpool 20220104
Loading data for: Liverpool 20220105
Loading data for: Liverpool 20220106
Loading data for: Liverpool 20220107
Loading data for: Liverpool 20220108
Loading data for: Liverpool 20220109
Loading data for: Liverpool 20220110
Loading data for: Liverpool 20220111
Loading data for: Liverpool 20220112
Loading data for: Liverpool 20220113
Loading data for: Liverpool 20220114
Loading data for: Liverpool 20220115
Loading data for: Liverpool 20220116
Loading data for: Liverpool 20220117
Loading data for: Liverpool 20220118
Loading data for: Liverpool 20220119
Loading data for: Liverpool 20220120
Loading data for: Liverpool 20220121
Loading data for: Liverpool 20220122
Loading data for: Liverpool 20220123
Loading data for: Liverpool 20220124
Loading data for: Liverpool 20220125
Loadi